In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras import optimizers

from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

from distutils.version import LooseVersion as LV 
import tensorflow as tf
from tensorflow.keras import __version__
from tensorflow.keras import backend as K

from IPython.display import SVG

import numpy as np

In [ ]:
data=np.load("../sample_data/detector_level.npy")
# data=np.load("../sample_data/parton_level.npy")

In [ ]:
# gpu=tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
data_inv = data[:,[0, 1, 2, 3, 12, 13, 14, 15, 8, 9, 10, 11, 4, 5, 6, 7, 16, 17]]
data = data[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
data_inv = np.array(data_inv)
label = np.ones(data.shape[0])
label_inv = np.zeros(data_inv.shape[0])
print (data.shape, data_inv.shape)

In [ ]:
x_train, y_train = np.concatenate((data[:90000], data_inv[:90000])),np.concatenate((label[:90000], label_inv[:90000]))
x_test, y_test = np.concatenate((data[90000:], data_inv[90000:])),np.concatenate((label[90000:], label_inv[90000:]))

idx = np.arange(x_train.shape[0])
np.random.shuffle(idx)
x_train,y_train = x_train[idx], y_train[idx]

print(x_train.shape, x_test.shape)

In [ ]:
model = Sequential()

model.add(Dense(512,input_shape=(18,), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

adam = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

print(model.summary())

In [ ]:
mcp_save = ModelCheckpoint('dnn_temp.h5', save_best_only=True, monitor='val_loss', mode='min')
es = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

In [ ]:
history = model.fit(x_train, y_train, batch_size=256,
          epochs=100, callbacks=[mcp_save, es],
          validation_split=0.1)

In [ ]:
model= load_model('dnn_temp.h5')
scores = model.evaluate(x_test, y_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100),model.metrics_names[0], scores[0])